In [1]:
# librerias

!pip3 install xmltodict

In [2]:
!pip3 install folium

In [3]:
!pip3 install pymongo

In [5]:
!pip3 install requests

In [6]:
# !pip3 install xmltodict
# !pip3 install folium
# !pip3 install pymongo
# !pip3 install requests
# !pip3 install regex

import warnings
warnings.simplefilter('ignore')  

import requests as req   # para get en la web
import xmltodict         # para convertir xml a diccionario
import re                # expresiones regulares

import folium                # mapa
from folium import plugins   # heatmap

import pandas as pd  # dataframe

import pymongo # mongodb

In [9]:
# extraccion 

def get_data():
    
    url='http://www.ssn.unam.mx/rss/ultimos-sismos.xml'
    
    data=req.get(url).content
    
    return xmltodict.parse(data)

In [15]:
get_data()['rss']['channel']['item'][0]['description']

'<p>Fecha:2020-02-29 05:54:21 (Hora de M&eacute;xico)<br/>Lat/Lon: 14.69/-93.88<br/>Profundidad: 19.0 km </p>'

In [ ]:
# https://regex101.com/

In [16]:
# transformacion

def get_dictio():
    
    d=get_data()
    
    res=[]
    
    for i in range(len(d['rss']['channel']['item'])):
        
        data_get=d['rss']['channel']['item'][i]['description'].split('>')[1:4]
        
        fecha=re.findall('\d+-\d+-\d+', data_get[0])[0]
        
        hora=re.findall('\d+:\d+:\d+', data_get[0])[0]
        
        lat=re.findall('-?\d+.\d+', data_get[1])[0]
        
        lng=re.findall('-?\d+.\d+', data_get[1])[1]
        
        prof=re.findall('\d+.\d+ \w+', data_get[2])[0]
        
        inten=float(d['rss']['channel']['item'][i]['title'].split(',')[0])
        
        
        data={'Fecha':fecha, 'Hora':hora, 'Latitud':lat,
              'Longitud':lng, 'Profundidad':prof, 'Intensidad':inten}
        
        res.append(data)
        
    return res

In [17]:
get_dictio()

[{'Fecha': '2020-02-29',
  'Hora': '05:54:21',
  'Latitud': '14.69',
  'Longitud': '-93.88',
  'Profundidad': '19.0 km',
  'Intensidad': 4.6},
 {'Fecha': '2020-02-29',
  'Hora': '04:58:57',
  'Latitud': '15.84',
  'Longitud': '-97.12',
  'Profundidad': '3.0 km',
  'Intensidad': 3.7},
 {'Fecha': '2020-02-29',
  'Hora': '04:26:37',
  'Latitud': '19.3',
  'Longitud': '-105.26',
  'Profundidad': '13.0 km',
  'Intensidad': 3.7},
 {'Fecha': '2020-02-29',
  'Hora': '03:59:25',
  'Latitud': '16.6',
  'Longitud': '-99.46',
  'Profundidad': '8.0 km',
  'Intensidad': 3.0},
 {'Fecha': '2020-02-29',
  'Hora': '03:56:36',
  'Latitud': '16.87',
  'Longitud': '-99.66',
  'Profundidad': '5.0 km',
  'Intensidad': 3.3},
 {'Fecha': '2020-02-29',
  'Hora': '03:41:19',
  'Latitud': '18.32',
  'Longitud': '-97.45',
  'Profundidad': '68.0 km',
  'Intensidad': 3.9},
 {'Fecha': '2020-02-29',
  'Hora': '03:26:13',
  'Latitud': '14.23',
  'Longitud': '-92.3',
  'Profundidad': '26.0 km',
  'Intensidad': 4.0},
 {'F

In [19]:
# carga base dastos de mongo

cliente=pymongo.MongoClient()
db=cliente.earthquake

In [20]:
def load_mongo():
    
    data=get_dictio()
    
    for e in data:
        print(e)
        db.ssnUnam.update(e, e, upsert=True)

In [21]:
load_mongo()

{'Fecha': '2020-02-29', 'Hora': '05:54:21', 'Latitud': '14.69', 'Longitud': '-93.88', 'Profundidad': '19.0 km', 'Intensidad': 4.6}
{'Fecha': '2020-02-29', 'Hora': '04:58:57', 'Latitud': '15.84', 'Longitud': '-97.12', 'Profundidad': '3.0 km', 'Intensidad': 3.7}
{'Fecha': '2020-02-29', 'Hora': '04:26:37', 'Latitud': '19.3', 'Longitud': '-105.26', 'Profundidad': '13.0 km', 'Intensidad': 3.7}
{'Fecha': '2020-02-29', 'Hora': '03:59:25', 'Latitud': '16.6', 'Longitud': '-99.46', 'Profundidad': '8.0 km', 'Intensidad': 3.0}
{'Fecha': '2020-02-29', 'Hora': '03:56:36', 'Latitud': '16.87', 'Longitud': '-99.66', 'Profundidad': '5.0 km', 'Intensidad': 3.3}
{'Fecha': '2020-02-29', 'Hora': '03:41:19', 'Latitud': '18.32', 'Longitud': '-97.45', 'Profundidad': '68.0 km', 'Intensidad': 3.9}
{'Fecha': '2020-02-29', 'Hora': '03:26:13', 'Latitud': '14.23', 'Longitud': '-92.3', 'Profundidad': '26.0 km', 'Intensidad': 4.0}
{'Fecha': '2020-02-29', 'Hora': '03:02:15', 'Latitud': '16.21', 'Longitud': '-98.11', 'P

In [24]:
# mapa

def load_map(query={'Intensidad':{'$gt':0}}):
    
    lst=list(db.ssnUnam.find(query))
    
    data=pd.DataFrame(lst)
    
    display(data.head())
    
    value_data=data[['Latitud', 'Longitud']].values
    
    mapa=folium.Map([19.42, -99.12], zoom_start=4)
    
    mapa.add_child(plugins.HeatMap(value_data, radius=15))
    
    #mapa.save('map_mex.html')
    
    display(mapa)

In [25]:
load_map()

,Fecha,Hora,Intensidad,Latitud,Longitud,Profundidad,_id
0,2019-10-28,09:08:45,4.0,26.04,-107.89,10.0 km,5db7805d3771a8f231530f47
1,2019-10-28,08:15:02,4.1,14.13,-93.34,19.0 km,5db7805d3771a8f231530f49
2,2019-10-28,04:59:59,3.8,17.9,-95.08,120.0 km,5db7805d3771a8f231530f4b
3,2019-10-28,04:55:15,3.4,16.69,-99.42,7.0 km,5db7805d3771a8f231530f4d
4,2019-10-28,04:34:54,3.3,16.3,-98.25,13.0 km,5db7805d3771a8f231530f4f


In [26]:
load_map({'Intensidad':{'$gt':5}})

,Fecha,Hora,Intensidad,Latitud,Longitud,Profundidad,_id
0,2019-12-23,09:47:55,5.8,13.63,-93.22,10.0 km,5e0145f01d3faec122d7e70a
1,2020-01-10,09:21:22,5.5,18.50,-105.41,12.0 km,5e18ae37dedeed93ea08b729
2,2020-01-11,08:22:03,5.1,16.26,-98.30,8.0 km,5e19f281dedeed93ea08ba32
3,2020-01-15,02:15:23,5.1,16.21,-98.06,6.0 km,5e1f22189f65f76062699457
4,2020-01-24,04:47:49,5.2,16.01,-97.20,27.0 km,5e2afec5c9b8672e2ec782b9


@gabocharles